# Распределение фильмов по уровням английского языка

In [9]:
! python3 -m pip install nltk pypdf2 plotly pandas scikit-learn catboost


[notice] A new release of pip available: 22.3.1 -> 23.1.2
[notice] To update, run: python3.11 -m pip install --upgrade pip


In [10]:
from pathlib import Path

import re
import pandas as pd
import plotly.graph_objects as go
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import balanced_accuracy_score
from catboost import CatBoostClassifier

import nltk
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from nltk.stem.wordnet import WordNetLemmatizer

In [11]:
nltk.download('wordnet')
nltk.download('stopwords')

[nltk_data] Downloading package wordnet to /Users/valery/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/valery/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

Для анализа уровней английского языка были сохранены списки слов по уровням, удалены пропущенные строки и информация о издании и страницах словаря


Загрузка данных:

In [89]:
files = list(Path('./data/English_scores/Subtitles_all/').glob('**/*.srt'))
files_labeled = [f for f in files if str(f).split('/')[-2] != 'Subtitles']
files_unlabeled = [f for f in files if str(f).split('/')[-2] == 'Subtitles']

Преобразуем данные, будем анализировать каждое слово в отдельности для этого читаем строки и отделяем кажое слово.

In [13]:
def read_data(path: Path | str) -> str:
    with open(str(path), "r", encoding='utf-8', errors='ignore') as f:
        data = f.read()
    
    return ' '.join([' '.join([i for i in f.split("\n")[2:]]) for f in data.split("\n\n")])

def read_dictionary(path: Path | str) -> str:
    with open(str(path), "r", encoding='utf-8', errors='ignore') as f:
        data = [re.findall(r'\w+', text)[0] for text in f.readlines()]
    
    return ' '.join(data)

Преобразуем размечанные и неразмечанные данные

In [14]:
text_labeled = [read_data(file) for file in files_labeled]
text_unlabeled = [read_data(file) for file in files_unlabeled]

In [15]:
labels = [str(file).split("/")[-2] for file in files_labeled]

Функция для процессинга данных, которая включает в себя приведение слов в нижний регистр, токенизацию, лемматизацию, стеминг и удаление стоп слов

In [16]:
def processing(text):
    text = text.strip().lower()
    text = nltk.RegexpTokenizer(r'\w+').tokenize(text)

    #lemmatizer = WordNetLemmatizer()
    #text = [lemmatizer.lemmatize(word) for word in text]

    stop_words = set(stopwords.words('english'))
    text = [word for word in text if word not in stop_words]

    porter = PorterStemmer()
    text = [porter.stem(word) for word in text]

    return text

In [17]:
text_labeled_proc = [processing(file) for file in text_labeled]
text_unlabeled_proc = [processing(file) for file in text_unlabeled]

Применим процессинг для словарей, чтобы удалить лишние символы и части речи

In [18]:
per_level_dict_proc = {
    key: dict.fromkeys(processing(read_dictionary(f"./data/{key}.txt")))
    for key in ["A1", "A2", "B1", "B2", "C1"]
}

Функция для расчета частоты встречаемости слов какого-либо уровня в данных

In [21]:
def distrib(words, dictionary):
    return {
        level: sum([word in dictionary[level] for word in words]) / len(words)
        for level in dictionary
    }


In [22]:
text_labeled_distrib = [distrib(words, per_level_dict_proc) for words in text_labeled_proc]
text_unlabeled_distrib = [distrib(words, per_level_dict_proc) for words in text_unlabeled_proc]

Расчет статистики содержания слов разных уровней, в фильмах какого-либо конкретного уровня

In [23]:
def avg_distrib(labels, distribs):
    stats = {}

    for file_level, distrib in zip(labels, distribs):
        if file_level not in stats:
            stats[file_level] = {level: [0, 0] for level in distrib}

        for level in distrib:
            stats[file_level][level][0] += distrib[level]
            stats[file_level][level][1] += 1
    
    _stats = {}
    for file_level in stats:
        _stats[file_level] = {}
        for level in stats[file_level]:
            _stats[file_level][level] = stats[file_level][level][0] / stats[file_level][level][1]

    return _stats

In [24]:
level_distrib = avg_distrib(labels, text_labeled_distrib)

Построим график распределения уровней слов в уровнях фильмов, чтобы определить можно ли только по распределению уровней слов в фильме определить, к какому уровню относиться весь фильм.

In [25]:
fig = go.Figure(data=[
    go.Bar(name=level, x=list(level_distrib[level].keys()), y=list(level_distrib[level].values()))
    for level in  sorted(level_distrib)
])


fig.update_layout(barmode='group', legend=dict(title='Sub level'))
fig.update_xaxes(title="English level")
fig.update_yaxes(title="% / 100")

fig.show()

Какие-то выводы о линейной зависимости сделать мы не можем, поэтому продолжим эксперемент без использования данной информации.

Объявим  и обучим лейбл энкодер

In [26]:
le = LabelEncoder().fit(labels)

Преобразуем данные и разделим их на train и valid

In [90]:
X, y = pd.DataFrame.from_records(text_labeled_distrib), le.transform(labels)
X_test = pd.DataFrame.from_records(text_unlabeled_distrib)

In [91]:
X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=0.2, stratify=y, random_state=1024)

Попробуем взять в качестве модели random forest и настроить гиперпараметры с использованием GridSearchCV

In [92]:
model = RandomForestClassifier(max_depth=2, random_state=1024).fit(X_train, y_train)

In [93]:
model.score(X_valid, y_valid)

0.8181818181818182

In [94]:
model_random_forest = RandomForestClassifier(random_state=1024)
parameters_2 = {
    'n_estimators': [5, 10, 15, 25, 50, 100], 
    'max_depth': [2, 3, 10, 15],
}
GSCV_forest = GridSearchCV(model_random_forest, parameters_2, scoring="balanced_accuracy").fit(X, y)
GSCV_forest.best_score_

0.6010281385281385

Попробуем взять в качестве модели также CatBoostClassifier

In [32]:
model_cat = CatBoostClassifier(eval_metric='Accuracy', depth=11, learning_rate=0.01, iterations=1000)
model_cat.fit(X_train, y_train, eval_set=(X_valid, y_valid))
model_cat.get_best_score()

0:	learn: 0.8615385	test: 0.5454545	best: 0.5454545 (0)	total: 71.7ms	remaining: 1m 11s
1:	learn: 0.8076923	test: 0.7575758	best: 0.7575758 (1)	total: 73.4ms	remaining: 36.6s
2:	learn: 0.8000000	test: 0.7272727	best: 0.7575758 (1)	total: 89ms	remaining: 29.6s
3:	learn: 0.8307692	test: 0.7272727	best: 0.7575758 (1)	total: 104ms	remaining: 25.8s
4:	learn: 0.8692308	test: 0.7575758	best: 0.7575758 (1)	total: 117ms	remaining: 23.2s
5:	learn: 0.8846154	test: 0.6969697	best: 0.7575758 (1)	total: 129ms	remaining: 21.4s
6:	learn: 0.8461538	test: 0.7272727	best: 0.7575758 (1)	total: 142ms	remaining: 20.1s
7:	learn: 0.8692308	test: 0.7272727	best: 0.7575758 (1)	total: 153ms	remaining: 18.9s
8:	learn: 0.8923077	test: 0.7575758	best: 0.7575758 (1)	total: 164ms	remaining: 18s
9:	learn: 0.9000000	test: 0.7575758	best: 0.7575758 (1)	total: 175ms	remaining: 17.3s
10:	learn: 0.9000000	test: 0.8181818	best: 0.8181818 (10)	total: 185ms	remaining: 16.6s
11:	learn: 0.8461538	test: 0.7575758	best: 0.8181818

{'learn': {'Accuracy': 1.0, 'MultiClass': 0.1894984154625019},
 'validation': {'Accuracy': 0.8181818181818182,
  'MultiClass': 0.5910064700669249}}

In [35]:
preds = model_cat.predict(X_valid)

In [36]:
balanced_accuracy_score(y_valid, preds.flatten())

0.7391774891774892

Сделаем предсказание для тестовой выборки

In [95]:
preds = model_cat.predict(X_test)

In [98]:
le.inverse_transform(preds.flatten())

array(['B2', 'B2', 'B2', 'B2', 'B2', 'B2', 'B2', 'B2', 'B2', 'A2', 'B2',
       'B2', 'B2', 'A2', 'B2', 'B2', 'B2', 'B2', 'B2', 'B2', 'B2', 'B2',
       'B2', 'C1', 'B2', 'B2', 'A2', 'B2', 'B2', 'B2', 'B2', 'B2', 'B2',
       'B2', 'A2', 'B2', 'B2', 'B2', 'B2', 'C1', 'A2', 'B2', 'B2', 'B2',
       'B2', 'A2', 'B2', 'B2', 'B2', 'B2', 'B2', 'B2', 'B2', 'B2', 'B2',
       'B2', 'B2', 'B2', 'B2', 'C1', 'B2', 'B2', 'B2', 'B2', 'C1', 'B2',
       'B2', 'A2', 'A2', 'B2', 'B2', 'A2', 'A2', 'B2', 'A2', 'B2', 'B2',
       'B2', 'B2', 'C1', 'B2', 'A2', 'B2', 'B2', 'C1', 'A2', 'B2', 'B2',
       'B2', 'A2', 'C1', 'B2', 'A2', 'B1', 'B2', 'C1', 'B2', 'C1', 'A2',
       'B2', 'A2', 'B2', 'B2', 'B2', 'B2', 'B2', 'B2', 'B2', 'B2', 'B2',
       'B2', 'B2', 'B2', 'B2', 'B2'], dtype='<U2')